# Just doing some neuroblastoma-specific stuff.

In particular, this notebook will run the mutation pipeline (or most of it) on a subset of the neuroblastoma (NB) cell lines for which we have not processed the BAM and BAI files we have. Most of these are samples from Sanger.

In addition, some subset of NB lines that we have CCLE data on were included in the CCLE2 paper, and already have results from running ABSOLUTE.

In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../../JKBio/')
sys.path.insert(0, '../')
from src.CCLE_postp_function import *
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np

In [ ]:
## Import up sound alert dependencies
from IPython.display import Audio, display

# play sound alert when function is called
def allDone():
    framerate = 4410
    play_time_seconds = 1

    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
    audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
    display(Audio(audio_data, rate=framerate, autoplay=True))

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
samplesetname = "NB_to_run"
refname_work = refnamespace + "/" + refworkspace

In [ ]:
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

## create sample set for NB lines to run

In [ ]:
# get list of all samples in the Sanger set
refwm.get_sample_sets()["samples"]["Sanger"]

In [ ]:
# get subset that haven't been run through the pipeline yet
# need to grab just these: 
nb_lines_to_run = pd.read_csv("../../NB-copy-loss/data/NB_lines_to_run.csv") # I used an R script to get this list.
nb_lines_to_run = nb_lines_to_run["x"].to_list()
nb_lines_to_run

In [ ]:
# create new sample set in Terra for these lines
refwm.update_sample_set(samplesetname, nb_lines_to_run)

In [ ]:
refwm.get_sample_sets()["samples"]["NB_to_run"] 

In [ ]:
# run the CGA characterization pipeline: will give MAF and ABSOLUTE as output
CGAmutations = refwm.get_config("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019")
CGAmutations

In [ ]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refname_work, submission_id)
allDone()